<a href="https://colab.research.google.com/github/rey2512/Model/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


In [2]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from timm import create_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

BASE_PATH = "/content/drive/MyDrive/1DeepFake/splitted_data"
TRAIN_REAL_PATH = os.path.join(BASE_PATH, "real/train")
TRAIN_FAKE_PATH = os.path.join(BASE_PATH, "fake/train")
VAL_REAL_PATH   = os.path.join(BASE_PATH, "real/val")
VAL_FAKE_PATH   = os.path.join(BASE_PATH, "fake/val")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

class DeepFakeDataset(Dataset):
    def __init__(self, file_list, labels, transform=None):
        self.file_list = file_list
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]
        cap = cv2.VideoCapture(path)
        ret, frame = cap.read()
        cap.release()
        if not ret:
            frame = np.zeros((299, 299, 3), dtype=np.uint8)
        frame = cv2.resize(frame, (299, 299))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if self.transform:
            frame = self.transform(frame)
        label = self.labels[idx]
        return frame, label


Using device: cuda


In [3]:
# Collect file paths
train_real = [os.path.join(TRAIN_REAL_PATH, f) for f in os.listdir(TRAIN_REAL_PATH) if f.endswith(".mp4")]
train_fake = [os.path.join(TRAIN_FAKE_PATH, f) for f in os.listdir(TRAIN_FAKE_PATH) if f.endswith(".mp4")]
val_real   = [os.path.join(VAL_REAL_PATH, f) for f in os.listdir(VAL_REAL_PATH) if f.endswith(".mp4")]
val_fake   = [os.path.join(VAL_FAKE_PATH, f) for f in os.listdir(VAL_FAKE_PATH) if f.endswith(".mp4")]

print("Train Real:", len(train_real), "Train Fake:", len(train_fake))
print("Val Real:", len(val_real), "Val Fake:", len(val_fake))

if len(train_real)==0 or len(train_fake)==0:
    raise Exception("❌ ERROR: Training folders are empty.")

# Combine all paths and labels
train_files = train_real + train_fake
train_labels = [0]*len(train_real) + [1]*len(train_fake)
val_files = val_real + val_fake
val_labels = [0]*len(val_real) + [1]*len(val_fake)

# Create Datasets
train_dataset = DeepFakeDataset(train_files, train_labels, transform)
val_dataset   = DeepFakeDataset(val_files, val_labels, transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Model
model = create_model("xception", pretrained=True, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss/len(train_loader):.4f}")

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Validation Accuracy: {100*correct/total:.2f}%")


Train Real: 280 Train Fake: 560
Val Real: 80 Val Fake: 160


/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth
Epoch [1/3] Loss: 0.5422
Validation Accuracy: 78.33%
Epoch [2/3] Loss: 0.1521
Validation Accuracy: 82.50%
Epoch [3/3] Loss: 0.0685
Validation Accuracy: 83.33%


In [4]:
import cv2
import torch
from torchvision import transforms
from PIL import Image

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model
model.load_state_dict(torch.load("/content/drive/MyDrive/1DeepFake/best_deepfake_detector2.pth"))
model.to(device)
model.eval()

# Transformation for Xception input
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Xception input size
    transforms.ToTensor(),
])

# Video path
video_path = "/content/drive/MyDrive/1DeepFake/test_video2.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fake_frames = 0
real_frames = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to PIL image
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    img = transform(img).unsqueeze(0).to(device)

    # Model prediction
    with torch.no_grad():
        output = model(img)
        prediction = torch.argmax(output, dim=1).item()

    if prediction == 0:
        real_frames += 1
    else:
        fake_frames += 1

cap.release()

# Overall Video Prediction
print(f"Total Frames: {frame_count}")
print(f"Real Frames: {real_frames}, Fake Frames: {fake_frames}")

if fake_frames > real_frames:
    print("❌ The video is FAKE")
else:
    print("✅ The video is REAL")


Total Frames: 443
Real Frames: 46, Fake Frames: 397
❌ The video is FAKE
